In [ ]:
# Parameters.
# Times Square replaces this cell with the user's parameters.
# So, don't put anything else here!

# day_obs values: TODAY, v, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
# day_obs = '2024-09-25' # 2024-12-06 2024-11-20 Value to use for local testing (Summit)
# 2024-11-30
# ComCam on-sky campaign: 24 October to 11 December 2024
# 2024-11-30: Exposures(372,2013) time: 9:18:47 + 1:49:23
# 2024-12-01: Exposures(510,1401) time: 5:42:08 + 2:32:42, has Good      <<<
# 2024-12-02: Exposures(738) time: 3:02:05, + LSSTComCam(423)
# 2024-12-07: Exposures(612) time: 3:13:06
# 2024-12-08: Exposures(738) time: 2:40:54
# 2024-12-10: Exposures(614) time: 2:45:17, has Good
# 2024-12-11: Exposures(612) time: 2:15:08
day_obs = "2024-12-02"  # TODO Change to 'YESTERDAY' and 'TODAY' to test with default before push
# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push
period = "2h"  # TODO change to 4h before push
verbose = "false"  # TODO change to false before push, else true
warning = "false"  # TODO change to false before push, else true

In [ ]:
# Read records from (almost) all sources
timer = ut.Timer()
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
    verbose=verbose,
    warning=warning,
    limit=5000,
    exclude_instruments=[],  # TODO change to empty list before push
)
allrep = AllReports(allsrc=allsrc)

In [ ]:
# IMPORT everything
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode, urlparse
import pandas as pd
from IPython.display import HTML

In [ ]:
# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    print(os.environ.get("EXTERNAL_INSTANCE_URL"))
    DEV_ENV = False
    dev_msg = (
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch. \n'
        "TODO: Make the need for this go away by getting Logging_and_Reporting installed in RSP."
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype > /dev/null 2>&1
else:
    DEV_ENV = True
    dev_msg = "Imported lsst.ts.logging_and_reporting from local packages."

In [ ]:
from lsst.ts.logging_and_reporting.all_sources import AllSources
from lsst.ts.logging_and_reporting.all_reports import AllReports
import lsst.ts.logging_and_reporting.utils as ut
import lsst.ts.logging_and_reporting.time_logs as tl
from lsst.ts.logging_and_reporting.reports import md, mdlist, mdpathlink, mdlink
from lsst.ts.logging_and_reporting.reports import html_draft, html_beta

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
# The default provided here is for local testing. Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", ut.Server.usdf
)  # TODO try with "usdf" before push (also "summit")

# Validate parameters, return usable ones
usable, error = ut.fallback_parameters(
    day_obs, number_of_days, period, verbose, warning
)
if error:
    print(error)

date = ut.get_datetime_from_dayobs_str(usable["day_obs"])
# date:  is EXCLUSIVE (upto, but not including)
days = usable["number_of_days"]
period = usable["period"]
verbose = usable["verbose"]
warning = usable["warning"]

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [ ]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
tlist = list()
tlist.append(f"## Showing data for {min_date.date()} to {max_date.date()}")
tlist.append(f"- **Instruments:** {instrum_str}")
tlist.append(f"- **Server:** {server}")
tlist.append(
    f'- Most recently run on: **{dt.datetime.now().replace(microsecond=0).isoformat(sep=" ")}**'
)
md("\n".join(tlist))

# Developer Only Section

Contains stuff only expected to be useful to developers.
This may also contain sections that have moved out of the user section.

## Overview 


In [ ]:
# Display overview of Report context
try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- min_dayobs={allsrc.min_dayobs!r}, max_dayobs={allsrc.max_dayobs!r}
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
- {have_consdb = }
- {dev_msg}
- {allsrc.nig_src.timeout=}
"""
)

## Data Status
This may be useful for diagnosing reports that give unexpected results.

In [ ]:
df = pd.DataFrame(allsrc.get_data_status())
table = df.to_html(index=False, render_links=True, escape=False)
display(HTML(table))

## Header for BETA section <font style="background-color:green; color:white; font-size:20px">BETA</font> 

## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Available Consolidated Database fields
It should be easy to add data to this report when it corresponds to any of the fields listed below. 

**Format:** *instrument/tablename/fieldname*

TODO: index by fieldname, multi-index columm = (Instrument, TableName), val = yes or NA

In [ ]:
verbose = True
if verbose:
    #! txt = '\n- '.join(allsrc.cdb_src.all_available_fields)
    #! md(f'- {txt}')
    table = ""
    table += "\n| instrument | tablename | fieldname |"
    table += "\n| :--------: | :-------: | :-------: |"
    for field in allsrc.cdb_src.all_available_fields:
        table += f'\n| {field.replace("/"," | ")} |'
    md(table)

## Section overviews moved here

In [ ]:
# Night Report Overview
allrep.nig_rep.overview()
# Exposure Report Overview
allrep.exp_rep.overview()
# Narrative Report Overview
allrep.nar_rep.overview()

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

In [ ]:
# Time Log
#! allrep.exp_rep.time_log_as_markdown()

# Finale

In [ ]:
print(f"Elapsed time (excluding code import): {timer.toc:.1f} seconds")
print(f"Finished {str(dt.datetime.now().replace(microsecond=0))} UTC")
allsrc